In [23]:
!pip install transformers

In [24]:
!pip install tensorboardx

In [25]:
!pip install torch==1.4.0+cpu torchvision==0.5.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [26]:
!pip install simpletransformers

In [0]:
from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import logging

In [0]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [0]:
import os
import collections
import pandas as pd

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data=pd.read_csv("/content/drive/My Drive/data/questions.csv",usecols=[3,5,6]) 
data=data[:5000]

if len(str(data['Problem Statement']).split(".",1))>1:
  data['Problem Statement']=data['Problem Statement'].str.split(".",1).str[1]

In [32]:
data['prob_sol'] = data['Problem Statement'] +' '+ data['Solution']
data.head()

,Problem Statement,Solution,tags,prob_sol
0,Read problems statements in Mandarin Chinese...,\n #include <cstring>\n #include <string...,['game'],Read problems statements in Mandarin Chinese...
1,Read problems statements in Mandarin Chinese...,\n #include <cstring>\n #include <string...,['game'],Read problems statements in Mandarin Chinese...
2,Read problems statements in Mandarin Chinese...,\n #include <iostream>\n #include <vecto...,['game'],Read problems statements in Mandarin Chinese...
3,Read problems statements in Mandarin Chinese...,\n #include <bits/stdc++.h> \n using na...,['game'],Read problems statements in Mandarin Chinese...
4,Read problems statements in Mandarin Chinese...,\n /***************************************...,['game'],Read problems statements in Mandarin Chinese...


In [33]:
data.head()
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = stopwords.words('english')
# data['Problem Statement'].apply(lambda x: [item for item in x if item not in stop])
# print("done1")
# data['Solution'].apply(lambda x: [item for item in x if item not in stop])
# print("done2")
data['prob_sol'].apply(lambda x: [item for item in x if item not in stop])
print("done3")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
done3


In [0]:
unique_labels=data.tags.unique()
num=len(unique_labels)

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
data['tags'].head()
print(type(data['tags'][0]))

<class 'str'>


In [0]:
from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data["prob_sol"].values, data["tags"].values, test_size=0.2, random_state=42)

In [0]:
model = MultiLabelClassificationModel(
    "xlnet",
    "xlnet-base-cased",

    
    num_labels=num,
    use_cuda=False,
    args={"reprocess_input_data": True, "overwrite_output_dir": True, "num_train_epochs": 1},
)

In [39]:
import numpy as np
print(y_train[0])
print(type(y_train))
import sklearn.preprocessing
a = y_train
print(a[0],a[1])
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
y1 = le.fit_transform(a)
print(y1[0],y1[1])
label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(max(y1)+1))
print(num)
b = label_binarizer.transform(y1)

print(b[0],b[1])
print(len(b[0]))
print(b.shape)
print(type(b))

['greedy']
<class 'numpy.ndarray'>
['greedy'] ['greedy']
12 12
23
[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0] [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
23
(4000, 23)
<class 'numpy.ndarray'>


In [40]:
print(type(X_train))
print(type(b))
# X_train.tolist()
b.tolist()
print(type(b[0]))
l=[]
import re
pattern = re.compile(r'\s+') 

for i in range(len(X_train)):
  l.append([re.sub(pattern, '',X_train[i]),b[i].tolist()])
# print(l[0])
train_df=pd.DataFrame(l)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [0]:
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:251: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


Running loss: 0.112655

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.028621

In [19]:
l=[]
a = y_test
# print(a[0],a[1])
from sklearn.preprocessing import LabelEncoder

y1 = le.fit_transform(a)
print(y1[0],y1[1])
# label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(max(y1)+1))
print(num)
b = label_binarizer.transform(y1)
for i in range(len(X_test)):
  l.append([re.sub(pattern, '',X_test[i]),b[i].tolist()])
# print(l[0])
eval_df=pd.DataFrame(l)

22 22
23


In [20]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
print(result)
print(model_outputs)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:668: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."



{'LRAP': 0.992237081339713, 'eval_loss': 0.018008361287415026}
[[0.00679919 0.01017659 0.00777258 ... 0.00489771 0.00901005 0.97149277]
 [0.00679919 0.01017659 0.00777258 ... 0.00489771 0.00901005 0.97149277]
 [0.00679919 0.01017659 0.00777258 ... 0.00489771 0.00901005 0.97149277]
 ...
 [0.02160067 0.02273116 0.01864196 ... 0.01984339 0.01838775 0.03395359]
 [0.01366162 0.02393527 0.01442645 ... 0.01584765 0.01238443 0.02330521]
 [0.0067992  0.0101766  0.00777259 ... 0.00489771 0.00901004 0.97149277]]


In [21]:
predictions, raw_outputs = model.predict(['''
Chef Ciel is playing a game with one of her colleagues.In this game, there are k piles of numbers. There are ni numbers in ith pile. In each move, player has to select a pile and finally a number from that pile. After that, all the numbers which are greater than or equal to chosen number will be removed from that pile. Players take turn alternatively. Player who is unable to move loses the game. 
Chef Ciel always make first move. But before making her move, she must choose either EVEN or ODD. If she chooses even then each time she selects a number it must be even and her opponent must select odd number in his/her turn. And vice versa for the other case. 
Please help Chef Ciel to make her choice. If neither choice can make her win game, print DONT PLAY otherwise EVEN or ODD depending on which choice she should make to win game. If you think then you will understand BOTH can not be answer.
Assume that both players play intelligently, if they can force win they will. 
 Input :  First line of test case contains t, number of test cases. Then t test cases follow. First line of each test case contains k, number of piles.  Then description of k piles follow. ith pile description contains number ni, then ni numbers follow in the same line.
Output :  Print EVEN, ODD or DONT PLAY accordingly.
Constraints :  t <= 1000, k <=100, ni <= 45. Numbers in pile will be non-negative and less than 2^31.
Sample Input :
2
2 
2 3 4
2 4 5
1
3 1 2 3
Sample Output :
DONT PLAY ODD
Explanation: 
In the first test case, if you have chosen ODD to play with, it means you will select only ODD numbers and your opponent may choose only EVEN numbers. So you have two options. Either select 3 from first pile or 5 from second pile. If 3 is selected, 3 and 4 are removed from first pile and in next move your opponents only move will be 4 from second pile removing every numbers thus you lose. If you had selected 5, then your opponents optimal move would be selecting 4 from first pile. Next you will select 3 and your opponent 4 leaving nothing for you. So if you choose ODD you will lose. Similarly if you choose EVEN you will lose as well. So the answer is DONT Play.
In the second case, if you choose ODD just select 1 and you will win. If you choose EVEN you have only one move that is 2, and in response your opponent will choose 1 and leave you nothing. Thus answer is ODD.'''])

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [22]:
# print(predictions)
print(raw_outputs)
y=label_binarizer.inverse_transform( raw_outputs, threshold=None)
y2=le.inverse_transform(y)
print(y2)
# list(LabelEncoder().inverse_transform([y1]))

[[0.01079385 0.008371   0.02516306 0.01700617 0.00565756 0.01970533
  0.00626744 0.01380369 0.02867823 0.02055322 0.00882878 0.00620659
  0.04096698 0.00794815 0.00599141 0.03655431 0.02121353 0.00530778
  0.00369297 0.02227653 0.00607185 0.01005958 0.02243079]]
["['greedy']"]
